In [1]:
# importing library
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd

In [2]:
# lists to store data
university_data = []
error_to_phase = []
error_to_phase2 = []

In [3]:
def getData(UniId, uCount):
    url = "https://nces.ed.gov/ipeds/datacenter/institutionprofile.aspx?unitId=" + str(UniId)
    page = requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')

    
    University_Data = soup.find_all('div', style="padding-top:7px; padding-bottom:7px;") # all the data related to university
    Information_Hadings = soup.find_all('div', style="float:left; padding-left:5px; padding-top:3px;") # phase university data
    Main_Headers = []  # list to store university information headers
    University_Information = {}
    for header in Information_Hadings:
        title_header = header.get_text(separator=' ').split()
        title_header = [x.capitalize() for x in title_header] # capitalizing all the first letters
        title_header = "_".join(title_header)
        Main_Headers.append(title_header) # adding to the Main_Headers list 
    # print(Main_Headers)
    # print(len(University_Data))
    count = 0
    for uni_data in University_Data:
        # print(uni_data)
        # print("count: ",count)
        content = uni_data.find_all('div', style="padding-bottom:30px;display:none;")
        Table_Title = []
        Table_Sub_Title = []
        for con in content:
            Table_Title = [x.get_text(separator=' ') for x in con.find_all('div', style="text-align:left; vertical-align:middle;font-size:12px;font-weight:bold;font-family:verdana, Arial;color:#4578ad;padding-top:17px; padding-bottom:14px;")]
            Table_Sub_Title = [x.get_text(separator=' ') for x in con.find_all('div', style="text-align:left; vertical-align:middle;font-size:12px;font-weight:bold;font-family:verdana, Arial;color:#4578ad;padding-top:30px; padding-bottom:14px;")]
            Table_Data = {} # palce to store all table data
            tables = con.find_all('table')
            # print("Number of Table: ", len(tables))
            table_count = 0
            for tab in tables:
                table_name = ""
                table_headings = [x for x in tab.find_all('tr', class_="idc_ip_header_tr")]
                main_table_headings = []
                for t_h in table_headings:
                    main_table_headings = [x.get_text(separator=' ') for x in t_h.find_all('td')]
                main_table_headings = ["_".join(x.split()) for x in main_table_headings]
                # print(main_table_headings)
                main_table_sub_headings = [x.get_text(separator=' ') for x in tab.find_all('td', class_="idc_ip_subheader_td")]
                tab_content = [x for x in tab.find_all('tr', class_="idc_ip_tr")]
                Table_Content = {}
                for tab_con in tab_content:
                    table_content = [x.get_text(separator=' ') for x in tab_con.find_all('td')]
                    topic = [x.strip().capitalize() for x in table_content[0].split()]

                    if Main_Headers[count] == "Net_Price":
                        topic = "".join(topic)
                    else:
                        topic = "_".join(topic)

                    if len(main_table_headings) == 0:
                        Table_Content[topic] = table_content[1].strip()
                        if Main_Headers[count] == "Student_Charges":
                            table_name = "Cost"
                            # print("table_name: ", table_name)
                        elif Main_Headers[count] == "Finance":
                            if table_count == 0:
                                table_name = "Core_Revenues_Per_FTE_Enrollment"
                            elif table_count == 1:
                                table_name = "Core_Expenses_Per_FTE_Enrollment"
                    else:
                        table_data = {}
                        if len(main_table_sub_headings) == 0:
                            for heading in range(len(main_table_headings)-1):    
                                try:
                                    table_data[main_table_headings[heading+1]] = table_content[heading+1]
                                except:
                                    table_data[main_table_headings[heading+1]] = ""
                            Table_Content[topic] = table_data
                            if Main_Headers[count] == "Completions":
                                # print(table_count)
                                if table_count == 0:
                                    table_name = "Total"
                                elif table_count == 1:
                                    table_name = "Men"
                                elif table_count == 2:
                                    table_name = "Women"
                                else:
                                    table_name = main_table_headings[0]
                                # print("table_name: ", table_name)
                            elif Main_Headers[count] == "Human_Resources":
                                if table_count == 1:
                                    table_name = "Number_Of_Full-Time_Staff"
                                else:
                                    table_name = main_table_headings[0]
                                    # print("table_name: ", table_name)
                            else:
                                if main_table_headings[0] != "":
                                    table_name = main_table_headings[0]
                                    # print("table_name: ", table_name)
                        else:
                            table_data[main_table_headings[1]] = table_content[1]
                            table_data[main_table_headings[2]] = table_content[2]
                            table_data[main_table_headings[3]] = {'Full_Time': table_content[3],'Part_Time':table_content[4]}
                            Table_Content[topic] = table_data
                            table_name = "Applications"
                            # print("table_name: ", table_name)
                
                # To put the informative table attributes
                if len(tables) == 1:
                    if table_name == "" :
                        table_name = Main_Headers[count]
                        # print("table_name: ", table_name)

                Table_Data[table_name] = Table_Content
                # Table_Data[] = Table_Content     
                # print(main_table_headings)
                # print(main_table_sub_headings)
                # print(Table_Content)
                # print("\n")        
            # print(Table_Title)
            # print(len(Table_Sub_Title), Table_Sub_Title)
                table_count = table_count + 1
                
            # print(Table_Data)
            University_Information[Main_Headers[count]] = Table_Data
        # print(University_Information)
        # print(University_Information)
        # University_Information[str(uCount)] = University_Information
        # print("\n\n\n\n\n\n")
        count = count + 1
    # print(University_Information)
    university_data.append({uCount: University_Information})
    # print("\n\n\n\n\n\n")

In [4]:
# # loading json file
# lodeJson = open('unitId.json', "r")
# uId_data = json.load(lodeJson).copy()
# total_uid = len(uId_data)
# scrap_data_count = 0
# for i in uId_data:
#     # print(i['UnitID']) # print university ID
#     print("Total University Count: ", total_uid)
#     perc = scrap_data_count/total_uid * 100
#     if perc < 1:
#         print("*")
#     # print("Completed: {} %".format(perc))
#     for i in range(int(perc)):
#         print("*")
#     print("Completed: {} %".format(perc))
#     print("Data Scraped: ", scrap_data_count)
#     print("Number of Error to Scrapped: ", len(error_to_phase))
#     uid = str(i['UnitID'])
#     try:
#         getData(uid,scrap_data_count+1)
#         scrap_data_count = scrap_data_count + 1
#     except:
#         error_to_phase.append(uid)
#         pass
#     # clear the screen
# #     clear = lambda: os.system('cls')
# #     clear()
#     # print(university_data)

In [5]:
# for j in error_to_phase:
#     try:
#         getData(j,scrap_data_count+1)
#         scrap_data_count = scrap_data_count + 1
#     except:
#         # error_to_phase2.append(j)
#         pass
# print("Unable to phase {} university data".format(len(error_to_phase2)))

In [6]:
# # getData(166027, 1)
# # print("********************************************************************")
# # getData(180203, 2)
# # print(university_data)

# # To export data in json file
# with open("scrapped_university_data.json", "w") as jsonwrite:
#     json.dump(university_data, jsonwrite)

In [7]:
lodeJson = open('unitId.json', "r")
uId_data = pd.read_json(lodeJson)
uId_data

,UnitID,Institution Name,
0,177834,A T Still University of Health Sciences,
1,180203,Aaniiih Nakoda College,
2,491464,ABC Adult School,
3,459523,ABC Beauty Academy,
4,485500,ABCO Technology,
...,...,...,...
6522,413820,Yukon Beauty College Inc,
6523,451237,Z Hair Academy,
6524,204255,Zane State College,
6525,458575,Zaytuna College,


In [8]:
uId_data["UnitID"]

0       177834
1       180203
2       491464
3       459523
4       485500
         ...  
6522    413820
6523    451237
6524    204255
6525    458575
6526    486248
Name: UnitID, Length: 6527, dtype: int64

In [9]:
len(uId_data)

6527

In [10]:
uId_data['UnitID'][0]

177834

In [11]:
total_uid = len(uId_data)
print("Total University Count: ", total_uid)
scrap_data_count = 0
for i in range(len(uId_data)):
  # print(i)
  perc = scrap_data_count/total_uid * 100
  print("Completed: {} %".format(perc))
  print("Data Scraped: ", scrap_data_count)
  # print(uId_data["UnitID"][i])
  try:
    getData(uId_data["UnitID"][i], scrap_data_count + 1)
    scrap_data_count = scrap_data_count + 1
  except:
    error_to_phase.append(uId_data["UnitID"][i])

Total University Count:  6527
Completed: 0.0 %
Data Scraped:  0
Completed: 0.015320974413972729 %
Data Scraped:  1
Completed: 0.030641948827945457 %
Data Scraped:  2
Completed: 0.04596292324191818 %
Data Scraped:  3
Completed: 0.061283897655890915 %
Data Scraped:  4
Completed: 0.07660487206986365 %
Data Scraped:  5
Completed: 0.09192584648383637 %
Data Scraped:  6
Completed: 0.1072468208978091 %
Data Scraped:  7
Completed: 0.12256779531178183 %
Data Scraped:  8
Completed: 0.13788876972575456 %
Data Scraped:  9
Completed: 0.1532097441397273 %
Data Scraped:  10
Completed: 0.1685307185537 %
Data Scraped:  11
Completed: 0.18385169296767273 %
Data Scraped:  12
Completed: 0.19917266738164546 %
Data Scraped:  13
Completed: 0.2144936417956182 %
Data Scraped:  14
Completed: 0.22981461620959093 %
Data Scraped:  15
Completed: 0.24513559062356366 %
Data Scraped:  16
Completed: 0.2604565650375364 %
Data Scraped:  17
Completed: 0.2757775394515091 %
Data Scraped:  18
Completed: 0.29109851386548186 %


Completed: 2.5126398038915276 %
Data Scraped:  164
Completed: 2.5279607783055003 %
Data Scraped:  165
Completed: 2.543281752719473 %
Data Scraped:  166
Completed: 2.5586027271334455 %
Data Scraped:  167
Completed: 2.573923701547418 %
Data Scraped:  168
Completed: 2.589244675961391 %
Data Scraped:  169
Completed: 2.604565650375364 %
Data Scraped:  170
Completed: 2.619886624789337 %
Data Scraped:  171
Completed: 2.6352075992033095 %
Data Scraped:  172
Completed: 2.650528573617282 %
Data Scraped:  173
Completed: 2.6658495480312547 %
Data Scraped:  174
Completed: 2.6811705224452274 %
Data Scraped:  175
Completed: 2.6964914968592 %
Data Scraped:  176
Completed: 2.7118124712731726 %
Data Scraped:  177
Completed: 2.7271334456871457 %
Data Scraped:  178
Completed: 2.7424544201011187 %
Data Scraped:  179
Completed: 2.7577753945150913 %
Data Scraped:  180
Completed: 2.773096368929064 %
Data Scraped:  181
Completed: 2.7884173433430366 %
Data Scraped:  182
Completed: 2.803738317757009 %
Data Scrap

Completed: 4.99463765895511 %
Data Scraped:  326
Completed: 5.009958633369082 %
Data Scraped:  327
Completed: 5.025279607783055 %
Data Scraped:  328
Completed: 5.040600582197028 %
Data Scraped:  329
Completed: 5.0559215566110005 %
Data Scraped:  330
Completed: 5.071242531024973 %
Data Scraped:  331
Completed: 5.086563505438946 %
Data Scraped:  332
Completed: 5.101884479852918 %
Data Scraped:  333
Completed: 5.117205454266891 %
Data Scraped:  334
Completed: 5.1325264286808645 %
Data Scraped:  335
Completed: 5.147847403094836 %
Data Scraped:  336
Completed: 5.16316837750881 %
Data Scraped:  337
Completed: 5.178489351922782 %
Data Scraped:  338
Completed: 5.193810326336755 %
Data Scraped:  339
Completed: 5.209131300750728 %
Data Scraped:  340
Completed: 5.2244522751647 %
Data Scraped:  341
Completed: 5.239773249578674 %
Data Scraped:  342
Completed: 5.2550942239926455 %
Data Scraped:  343
Completed: 5.270415198406619 %
Data Scraped:  344
Completed: 5.285736172820591 %
Data Scraped:  345
C

Completed: 7.522598437260609 %
Data Scraped:  491
Completed: 7.537919411674583 %
Data Scraped:  492
Completed: 7.5532403860885555 %
Data Scraped:  493
Completed: 7.568561360502528 %
Data Scraped:  494
Completed: 7.5838823349165 %
Data Scraped:  495
Completed: 7.599203309330474 %
Data Scraped:  496
Completed: 7.614524283744446 %
Data Scraped:  497
Completed: 7.629845258158419 %
Data Scraped:  498
Completed: 7.645166232572391 %
Data Scraped:  499
Completed: 7.660487206986365 %
Data Scraped:  500
Completed: 7.675808181400337 %
Data Scraped:  501
Completed: 7.691129155814309 %
Data Scraped:  502
Completed: 7.7064501302282835 %
Data Scraped:  503
Completed: 7.721771104642255 %
Data Scraped:  504
Completed: 7.737092079056228 %
Data Scraped:  505
Completed: 7.7524130534702005 %
Data Scraped:  506
Completed: 7.767734027884174 %
Data Scraped:  507
Completed: 7.783055002298147 %
Data Scraped:  508
Completed: 7.798375976712118 %
Data Scraped:  509
Completed: 7.813696951126092 %
Data Scraped:  510

Completed: 10.05055921556611 %
Data Scraped:  656
Completed: 10.065880189980083 %
Data Scraped:  657
Completed: 10.081201164394056 %
Data Scraped:  658
Completed: 10.096522138808028 %
Data Scraped:  659
Completed: 10.111843113222001 %
Data Scraped:  660
Completed: 10.127164087635974 %
Data Scraped:  661
Completed: 10.142485062049946 %
Data Scraped:  662
Completed: 10.157806036463919 %
Data Scraped:  663
Completed: 10.173127010877892 %
Data Scraped:  664
Completed: 10.188447985291866 %
Data Scraped:  665
Completed: 10.203768959705837 %
Data Scraped:  666
Completed: 10.21908993411981 %
Data Scraped:  667
Completed: 10.234410908533782 %
Data Scraped:  668
Completed: 10.249731882947756 %
Data Scraped:  669
Completed: 10.265052857361729 %
Data Scraped:  670
Completed: 10.2803738317757 %
Data Scraped:  671
Completed: 10.295694806189672 %
Data Scraped:  672
Completed: 10.311015780603647 %
Data Scraped:  673
Completed: 10.32633675501762 %
Data Scraped:  674
Completed: 10.341657729431592 %
Data

...

Completed: 88.66247893366018 %
Data Scraped:  5787
Completed: 88.67779990807415 %
Data Scraped:  5788
Completed: 88.69312088248813 %
Data Scraped:  5789
Completed: 88.7084418569021 %
Data Scraped:  5790
Completed: 88.72376283131607 %
Data Scraped:  5791
Completed: 88.73908380573005 %
Data Scraped:  5792
Completed: 88.75440478014401 %
Data Scraped:  5793
Completed: 88.769725754558 %
Data Scraped:  5794
Completed: 88.78504672897196 %
Data Scraped:  5795
Completed: 88.80036770338594 %
Data Scraped:  5796
Completed: 88.81568867779991 %
Data Scraped:  5797
Completed: 88.83100965221388 %
Data Scraped:  5798
Completed: 88.84633062662786 %
Data Scraped:  5799
Completed: 88.86165160104183 %
Data Scraped:  5800
Completed: 88.87697257545581 %
Data Scraped:  5801
Completed: 88.89229354986978 %
Data Scraped:  5802
Completed: 88.90761452428374 %
Data Scraped:  5803
Completed: 88.92293549869773 %
Data Scraped:  5804
Completed: 88.9382564731117 %
Data Scraped:  5805
Completed: 88.95357744752566 %
Data

Completed: 91.14447678872376 %
Data Scraped:  5949
Completed: 91.15979776313775 %
Data Scraped:  5950
Completed: 91.17511873755171 %
Data Scraped:  5951
Completed: 91.19043971196568 %
Data Scraped:  5952
Completed: 91.20576068637966 %
Data Scraped:  5953
Completed: 91.22108166079363 %
Data Scraped:  5954
Completed: 91.2364026352076 %
Data Scraped:  5955
Completed: 91.25172360962156 %
Data Scraped:  5956
Completed: 91.26704458403555 %
Data Scraped:  5957
Completed: 91.28236555844951 %
Data Scraped:  5958
Completed: 91.29768653286348 %
Data Scraped:  5959
Completed: 91.31300750727746 %
Data Scraped:  5960
Completed: 91.32832848169143 %
Data Scraped:  5961
Completed: 91.34364945610541 %
Data Scraped:  5962
Completed: 91.35897043051938 %
Data Scraped:  5963
Completed: 91.37429140493335 %
Data Scraped:  5964
Completed: 91.38961237934733 %
Data Scraped:  5965
Completed: 91.4049333537613 %
Data Scraped:  5966
Completed: 91.42025432817528 %
Data Scraped:  5967
Completed: 91.43557530258924 %
Da

Completed: 93.62647464378735 %
Data Scraped:  6111
Completed: 93.64179561820131 %
Data Scraped:  6112
Completed: 93.65711659261528 %
Data Scraped:  6113
Completed: 93.67243756702926 %
Data Scraped:  6114
Completed: 93.68775854144323 %
Data Scraped:  6115
Completed: 93.70307951585721 %
Data Scraped:  6116
Completed: 93.71840049027118 %
Data Scraped:  6117
Completed: 93.73372146468515 %
Data Scraped:  6118
Completed: 93.74904243909913 %
Data Scraped:  6119
Completed: 93.7643634135131 %
Data Scraped:  6120
Completed: 93.77968438792708 %
Data Scraped:  6121
Completed: 93.79500536234104 %
Data Scraped:  6122
Completed: 93.81032633675501 %
Data Scraped:  6123
Completed: 93.82564731116899 %
Data Scraped:  6124
Completed: 93.84096828558296 %
Data Scraped:  6125
Completed: 93.85628925999694 %
Data Scraped:  6126
Completed: 93.87161023441091 %
Data Scraped:  6127
Completed: 93.88693120882489 %
Data Scraped:  6128
Completed: 93.90225218323886 %
Data Scraped:  6129
Completed: 93.91757315765282 %
D

Completed: 96.10847249885093 %
Data Scraped:  6273
Completed: 96.1237934732649 %
Data Scraped:  6274
Completed: 96.13911444767888 %
Data Scraped:  6275
Completed: 96.15443542209285 %
Data Scraped:  6276
Completed: 96.16975639650683 %
Data Scraped:  6277
Completed: 96.1850773709208 %
Data Scraped:  6278
Completed: 96.20039834533476 %
Data Scraped:  6279
Completed: 96.21571931974874 %
Data Scraped:  6280
Completed: 96.23104029416271 %
Data Scraped:  6281
Completed: 96.24636126857669 %
Data Scraped:  6282
Completed: 96.26168224299066 %
Data Scraped:  6283
Completed: 96.27700321740463 %
Data Scraped:  6284
Completed: 96.29232419181861 %
Data Scraped:  6285
Completed: 96.30764516623258 %
Data Scraped:  6286
Completed: 96.32296614064654 %
Data Scraped:  6287
Completed: 96.33828711506052 %
Data Scraped:  6288
Completed: 96.35360808947449 %
Data Scraped:  6289
Completed: 96.36892906388846 %
Data Scraped:  6290
Completed: 96.38425003830243 %
Data Scraped:  6291
Completed: 96.39957101271641 %
Da

Completed: 98.59047035391451 %
Data Scraped:  6435
Completed: 98.60579132832848 %
Data Scraped:  6436
Completed: 98.62111230274245 %
Data Scraped:  6437
Completed: 98.63643327715643 %
Data Scraped:  6438
Completed: 98.6517542515704 %
Data Scraped:  6439
Completed: 98.66707522598436 %
Data Scraped:  6440
Completed: 98.68239620039834 %
Data Scraped:  6441
Completed: 98.69771717481231 %
Data Scraped:  6442
Completed: 98.71303814922629 %
Data Scraped:  6443
Completed: 98.72835912364026 %
Data Scraped:  6444
Completed: 98.74368009805423 %
Data Scraped:  6445
Completed: 98.75900107246821 %
Data Scraped:  6446
Completed: 98.77432204688218 %
Data Scraped:  6447
Completed: 98.78964302129616 %
Data Scraped:  6448
Completed: 98.80496399571012 %
Data Scraped:  6449
Completed: 98.82028497012409 %
Data Scraped:  6450
Completed: 98.83560594453807 %
Data Scraped:  6451
Completed: 98.85092691895204 %
Data Scraped:  6452
Completed: 98.86624789336602 %
Data Scraped:  6453
Completed: 98.88156886777999 %
D

In [44]:
print("Error in scrapping university data count is : ", len(error_to_phase))
print(error_to_phase)

Error in scrapping university data count is :  11
[492032, 460446, 200208, 401481, 413413, 488590, 452540, 451033, 494454, 201982, 475237]


In [1]:
print("Scrapping Again")
scrap_data_counts = scrap_data_count
# starting 
for etp in error_to_phase:
  # print(i)
  perc = scrap_data_counts/total_uid * 100
  print("Completed: {} %".format(perc))
  print("Data Scraped: ", scrap_data_counts)
  # print(uId_data["UnitID"][i])
  try:
    getData(etp, scrap_data_counts + 1)
    scrap_data_count = scrap_data_counts + 1
  except:
    error_to_phase2.append(etp)

Scrapping Again


NameError: name 'scrap_data_count' is not defined

In [42]:
print("Error in scrapping university data count is : ", len(error_to_phase2))
print(error_to_phase2)

Error in scrapping university data count is :  56
[492032, 460446, 488590, 452540, 451033, 494454, 201982, 475237, 492032, 460446, 488590, 452540, 451033, 494454, 201982, 475237, 492032, 460446, 488590, 452540, 451033, 494454, 201982, 475237, 492032, 460446, 488590, 452540, 451033, 494454, 201982, 475237, 492032, 460446, 488590, 452540, 451033, 494454, 201982, 475237, 492032, 460446, 488590, 452540, 451033, 494454, 201982, 475237, 492032, 460446, 488590, 452540, 451033, 494454, 201982, 475237]


In [46]:
print("Exporting scrapped data into JSON file")
# To export data in json file
with open("scrapped_university_data1.json", "w") as jsonwrite:
    json.dump(university_data, jsonwrite, indent=4)
print("Completed!")

Exporting scrapped data into JSON file
Completed!
